In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

%matplotlib inline

Creating categories for matching based on kg and lm uom_type

In [2]:
df = pd.read_excel('dec_iNet_cleaned.xlsx',sheet_name='Sheet1')

In [3]:
df.head(2)

,Bill no,Bill date,Bill sender id,Bill sender name,Load no,TO no,Country TO consignor,Zip TO consignor,City TO consignor,Name TO consignor,Country TO consignee,Zip TO consignee,City TO consignee,Name TO consignee,Country leg consignor,Zip leg consignor,City leg consignor,Name leg consignor,Country leg consignee,Zip leg consignee,City leg consignee,Name leg consignee,Processing date,Pickup date,Weight TO [kg],Volume TO [m3],Loading meter leg,Transport mode leg,Cost type,Cost desc,Cost rate,Amount,uom_type,uom_value,pricing_category
0,27216,10.12.2021,SCAC81,Jan De Rijk,1708024,23020651,NL,2231,Rijnsburg,Alutech Rijnsburg,NL,8041 AL,Zwolle,Scania Production Zwolle,NL,2231,Rijnsburg,Alutech Rijnsburg,NL,8041 AL,Zwolle,Scania Production Zwolle,08.12.2021,08.12.2021,2560.0,78.72,13.6,FTL,SCAPAD,Only Inbound Invoicing Users - Price Adjustment,NaN,1206.85,not_lm_kg,0.0000,not_lm_kg
1,27034,03.12.2021,SCAC10-13,DSV ES,1695101,30239604,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,12540,VILLARREAL (CASTELLÓN),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,12540,VILLARREAL (CASTELLÓN),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",29.11.2021,24.11.2021,12845.0,69.75,11.8,LTL,SCAMAT,Maut,23226.8835 kg.,92.90,kg,23226.8835,23000_24499_kg


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341430 entries, 0 to 341429
Data columns (total 35 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Bill no                341430 non-null  object 
 1   Bill date              341430 non-null  object 
 2   Bill sender id         341430 non-null  object 
 3   Bill sender name       341430 non-null  object 
 4   Load no                341430 non-null  int64  
 5   TO no                  341430 non-null  object 
 6   Country TO consignor   341430 non-null  object 
 7   Zip TO consignor       341430 non-null  object 
 8   City TO consignor      341430 non-null  object 
 9   Name TO consignor      341430 non-null  object 
 10  Country TO consignee   341430 non-null  object 
 11  Zip TO consignee       341430 non-null  object 
 12  City TO consignee      341430 non-null  object 
 13  Name TO consignee      341430 non-null  object 
 14  Country leg consignor  341430 non-nu

In [5]:
#df['uom_value'] = pd.to_numeric(df['uom_value'],errors='coerce')

In [6]:
df.groupby("uom_type", as_index=False).agg(LoadCount=('Load no','nunique'),
                                     ToCount=('TO no','nunique'),
                                     TotalPaid=('Amount','sum'),
                                     v=('uom_value','sum')).sort_values(by='TotalPaid',ascending=False)

,uom_type,LoadCount,ToCount,TotalPaid,v
1,lm,8570,49728,9848744.59,1.028397e+06
2,not_lm_kg,26671,73537,3312973.03,0.000000e+00
0,kg,13917,40749,2771097.36,3.672159e+08


In [7]:
df['uom_value'] = df['uom_value'].fillna(0)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341430 entries, 0 to 341429
Data columns (total 35 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Bill no                341430 non-null  object 
 1   Bill date              341430 non-null  object 
 2   Bill sender id         341430 non-null  object 
 3   Bill sender name       341430 non-null  object 
 4   Load no                341430 non-null  int64  
 5   TO no                  341430 non-null  object 
 6   Country TO consignor   341430 non-null  object 
 7   Zip TO consignor       341430 non-null  object 
 8   City TO consignor      341430 non-null  object 
 9   Name TO consignor      341430 non-null  object 
 10  Country TO consignee   341430 non-null  object 
 11  Zip TO consignee       341430 non-null  object 
 12  City TO consignee      341430 non-null  object 
 13  Name TO consignee      341430 non-null  object 
 14  Country leg consignor  341430 non-nu

In [9]:
def categorize_uom(row):
    if row['uom_type'] == 'kg':
        if row['uom_value'] <= 49:
            return '0_49_kg'
        elif row['uom_value'] <= 99:
            return '50_99_kg'
        elif row['uom_value'] <= 149:
            return '100_149_kg'
        elif row['uom_value'] <= 199:
            return '150_199_kg'
        elif row['uom_value'] <= 299:
            return '200_299_kg'
        elif row['uom_value'] <= 399:
            return '300_399_kg'
        elif row['uom_value'] <= 499:
            return '400_499_kg'
        elif row['uom_value'] <= 599:
            return '500_599_kg'
        elif row['uom_value'] <= 699:
            return '600_699_kg'
        elif row['uom_value'] <= 799:
            return '700_799_kg'
        elif row['uom_value'] <= 899:
            return '800_899_kg'
        elif row['uom_value'] <= 999:
            return '900_999_kg'
        elif row['uom_value'] <= 1099:
            return '1000_1099_kg'
        elif row['uom_value'] <= 1199:
            return '1100_1199_kg'
        elif row['uom_value'] <= 1299:
            return '1200_1299_kg'
        elif row['uom_value'] <= 1399:
            return '1300_1399_kg'
        elif row['uom_value'] <= 1499:
            return '1400_1499_kg'
        elif row['uom_value'] <= 1749:
            return '1500_1749_kg'
        elif row['uom_value'] <= 1999:
            return '1750_1999_kg'
        elif row['uom_value'] <= 2249:
            return '2000_2249_kg'
        elif row['uom_value'] <= 2499:
            return '2250_2499_kg'
        elif row['uom_value'] <= 2999:
            return '2500_2999_kg'
        elif row['uom_value'] <= 3499:
            return '3000_3499_kg'
        elif row['uom_value'] <= 3999:
            return '3500_3999_kg'
        elif row['uom_value'] <= 4499:
            return '4000_4499_kg'
        elif row['uom_value'] <= 4999:
            return '4500_4999_kg'
        elif row['uom_value'] <= 5999:
            return '5000_5999_kg'
        elif row['uom_value'] <= 6999:
            return '6000_6999_kg'
        elif row['uom_value'] <= 7999:
            return '7000_7999_kg'
        elif row['uom_value'] <= 8999:
            return '8000_8999_kg'
        elif row['uom_value'] <= 9999:
            return '9000_9999_kg'
        elif row['uom_value'] <= 10999:
            return '10000_10999_kg'
        elif row['uom_value'] <= 11999:
            return '11000_11999_kg'
        elif row['uom_value'] <= 12999:
            return '12000_12999_kg'
        elif row['uom_value'] <= 13999:
            return '13000_13999_kg'
        elif row['uom_value'] <= 14999:
            return '14000_14999_kg'
        elif row['uom_value'] <= 15999:
            return '15000_15999_kg'
        elif row['uom_value'] <= 16999:
            return '16000_16999_kg'
        elif row['uom_value'] <= 17999:
            return '17000_17999_kg'
        elif row['uom_value'] <= 18999:
            return '18000_18999_kg'
        elif row['uom_value'] <= 19999:
            return '19000_19999_kg'
        elif row['uom_value'] <= 21499:
            return '20000_21499_kg'
        elif row['uom_value'] <= 22999:
            return '21500_22999_kg'
        elif row['uom_value'] <= 24499:
            return '23000_24499_kg'
        elif row['uom_value'] > 24499:
            return 'FTL_kg'

# type of cost calculation
df["pricing_category"] = df.apply(categorize_uom, axis=1)

In [10]:
df[df['uom_type']=='kg']

,Bill no,Bill date,Bill sender id,Bill sender name,Load no,TO no,Country TO consignor,Zip TO consignor,City TO consignor,Name TO consignor,Country TO consignee,Zip TO consignee,City TO consignee,Name TO consignee,Country leg consignor,Zip leg consignor,City leg consignor,Name leg consignor,Country leg consignee,Zip leg consignee,City leg consignee,Name leg consignee,Processing date,Pickup date,Weight TO [kg],Volume TO [m3],Loading meter leg,Transport mode leg,Cost type,Cost desc,Cost rate,Amount,uom_type,uom_value,pricing_category
1,27034,03.12.2021,SCAC10-13,DSV ES,1695101,30239604,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,12540,VILLARREAL (CASTELLÓN),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,12540,VILLARREAL (CASTELLÓN),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",29.11.2021,24.11.2021,12845.0,69.750,11.800,LTL,SCAMAT,Maut,23226.8835 kg.,92.90,kg,23226.8835,23000_24499_kg
2,27034,03.12.2021,SCAC10-13,DSV ES,1695100,30239615,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,29.11.2021,24.11.2021,13812.0,72.853,13.262,LTL,SCAMAT,Maut,24499.0 kg.,92.90,kg,24499.0000,23000_24499_kg
3,27034,03.12.2021,SCAC10-13,DSV ES,1700008,30240209,ES,19004,GUADALAJARA,"NEOVIA LOGISTICS SERVICES SPAIN, S.A.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,19004,GUADALAJARA,"NEOVIA LOGISTICS SERVICES SPAIN, S.A.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,03.12.2021,30.11.2021,14772.0,71.332,13.600,LTL,SCAMAT,Maut,24499.0 kg.,92.90,kg,24499.0000,23000_24499_kg
4,27034,03.12.2021,SCAC10-13,DSV ES,1699921,30240157,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,03.12.2021,30.11.2021,8654.0,75.361,13.304,LTL,SCAMAT,Maut,24499.0 kg.,92.90,kg,24499.0000,23000_24499_kg
5,27034,03.12.2021,SCAC10-13,DSV ES,1699922,30240166,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,50290,EPILA ZARAGOZA,KONGSBERG ACTUATION SYSTEM S.L.,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,50290,EPILA ZARAGOZA,KONGSBERG ACTUATION SYSTEM S.L.,03.12.2021,30.11.2021,18809.0,76.086,13.600,LTL,SCAMAT,Maut,24499.0 kg.,92.90,kg,24499.0000,23000_24499_kg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295956,27493,31.12.2021,SCAC31-1,EKOL,1717902,10164278,TR,41420,CAYIROVA-GEBZE,KALE OTO RADYATOR SANAYI VE TICARET S A,NL,8061,Hasselt,Scania CV AB,TR,41420,CAYIROVA-GEBZE,KALE OTO RADYATOR SANAYI VE TICARET S A,TR,34935,ISTANBUL,EKOL X-DOCK GEBZE,28.12.2021,21.12.2021,284.0,3.842,0.814,LTL,SCAFRE,Freight Charges,1424.5 kg.,105.00,kg,1424.5000,1400_1499_kg
295957,27493,31.12.2021,SCAC31-1,EKOL,1717053,23039328,TR,35410,GAZIEMIR-IZMIR,EGE FREN SANAYY VE TICARET A.,BE,3660,OUDSBERGEN,Scania CV AB,TR,35410,GAZIEMIR-IZMIR,EGE FREN SANAYY VE TICARET A.,TR,34935,ISTANBUL,EKOL X-DOCK GEBZE,29.12.2021,20.12.2021,649.0,1.192,0.350,LTL,SCAFRE,Freight Charges,649.0 kg.,400.00,kg,649.0000,600_699_kg
295958,27493,31.12.2021,SCAC31-1,EKOL,1717054,10163733,TR,41490,KOCAELI,KANCA EL ALETLERI DÖVME CELIK VE MAKINA SANAYI...,NL,8061,Hasselt,Scania CV AB,TR,41490,KOCAELI,KANCA EL ALETLERI DÖVME CELIK VE MAKINA SANAYI...,TR,34935,ISTANBUL,EKOL X-DOCK GEBZE,29.12.2021,20.12.2021,2099.0,24.330,4.379,LTL,SCAFRE,Freight Charges,7663.25 kg.,155.66,kg,7663.2500,7000_7999_kg
295959,27493,31.12.2021,SCAC31-1,EKOL,1717056,10164242,TR,16131,Bursa,Beltan Vibracoustic,BE,3660,OUDSBERGEN,Scania CV AB,TR,16131,Bursa,Beltan Vibracoustic,TR,34935,ISTANBUL,EKOL X-DOCK GEBZE,29.12.2021,20.12.2021,1081.0,14.406,4.880,LTL,SCAFRE,Freight Charges,23749.25 kg.,58.18,kg,23749.2500,23000_24499_kg


In [11]:
def categorize_uom_lm(row):
    if row['uom_type'] == 'lm':
        if row['uom_value'] <= 0.5:
            return '0.5_lm'
        elif row['uom_value'] <= 1:
            return '1_lm'
        elif row['uom_value'] <= 2:
            return '2_lm'
        elif row['uom_value'] <= 3:
            return '3_lm'
        elif row['uom_value'] <= 4:
            return '4_lm'
        elif row['uom_value'] <= 5:
            return '5_lm'
        elif row['uom_value'] <= 6:
            return '6_lm'
        elif row['uom_value'] <= 7:
            return '7_lm'
        elif row['uom_value'] <= 8:
            return '8_lm'
        elif row['uom_value'] <= 9:
            return '9_lm'
        elif row['uom_value'] <= 10:
            return '10_lm'
        elif row['uom_value'] <= 11:
            return '11_lm'
        elif row['uom_value'] <= 12:
            return '12_lm'
        elif row['uom_value'] <= 13.6:
            return '13.6_lm'
        elif row['uom_value'] > 13.6:
            return 'FTL_lm'
    elif row['uom_type'] == 'kg':
        return row["pricing_category"]
    else:
        return 'not_lm_kg'
        
        
        
# type of cost calculation
df["pricing_category"] = df.apply(categorize_uom_lm, axis=1)

In [12]:
#df["pricing_category"].value_counts()
#[df['uom_type']=='lm']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341430 entries, 0 to 341429
Data columns (total 35 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Bill no                341430 non-null  object 
 1   Bill date              341430 non-null  object 
 2   Bill sender id         341430 non-null  object 
 3   Bill sender name       341430 non-null  object 
 4   Load no                341430 non-null  int64  
 5   TO no                  341430 non-null  object 
 6   Country TO consignor   341430 non-null  object 
 7   Zip TO consignor       341430 non-null  object 
 8   City TO consignor      341430 non-null  object 
 9   Name TO consignor      341430 non-null  object 
 10  Country TO consignee   341430 non-null  object 
 11  Zip TO consignee       341430 non-null  object 
 12  City TO consignee      341430 non-null  object 
 13  Name TO consignee      341430 non-null  object 
 14  Country leg consignor  341430 non-nu

In [13]:
df[df['uom_type']=='not_lm_kg'][['uom_type','pricing_category']].value_counts()


uom_type   pricing_category
not_lm_kg  not_lm_kg           200568
dtype: int64

In [14]:
df2 = df.groupby(['Load no','Cost type'],as_index=False).agg(tr_mode_Type = ('Transport mode leg','nunique'),
                         uomType_co = ('uom_type','nunique'),
                         uomvalue_co = ('uom_value','nunique'))




df2[df2['tr_mode_Type']>1]

,Load no,Cost type,tr_mode_Type,uomType_co,uomvalue_co


In [15]:
#df.to_excel('dec_iNet_cleaned2.xlsx', index=False)

---
Creating the groups as per grouping structure layout

In [16]:
df.groupby(['Load no','Country TO consignor','City TO consignor','Country TO consignee','City TO consignee','Cost desc'], 
           as_index=False).agg(amount=('Amount','sum'))

,Load no,Country TO consignor,City TO consignor,Country TO consignee,City TO consignee,Cost desc,amount
0,1428428,SE,Järna,SE,KALIX,Freight Charges,1355.56
1,1428428,SE,Järna,SE,KALIX,Fuel Surcharges,6.96
2,1535615,DE,BAD KISSINGEN,NL,Hasselt,Waiting time (consignor),42.20
3,1535615,DE,BAD KISSINGEN,NL,Hasselt,Waiting time Supplier,89.44
4,1535615,DE,BAD KISSINGEN,NL,Zwolle,Waiting time (consignor),53.80
...,...,...,...,...,...,...,...
254238,1721306,SE,FALUN,SE,SODERTALJE,Freight Charges,101.41
254239,1721306,SE,FALUN,SE,SODERTALJE,Fuel Surcharges,6.37
254240,1721385,FR,QUIMPER,FR,LIEUSAINT,Freight Charges,497.70
254241,1721385,FR,QUIMPER,FR,LIEUSAINT,Fuel Surcharges,17.52


In [17]:
df.head()

,Bill no,Bill date,Bill sender id,Bill sender name,Load no,TO no,Country TO consignor,Zip TO consignor,City TO consignor,Name TO consignor,Country TO consignee,Zip TO consignee,City TO consignee,Name TO consignee,Country leg consignor,Zip leg consignor,City leg consignor,Name leg consignor,Country leg consignee,Zip leg consignee,City leg consignee,Name leg consignee,Processing date,Pickup date,Weight TO [kg],Volume TO [m3],Loading meter leg,Transport mode leg,Cost type,Cost desc,Cost rate,Amount,uom_type,uom_value,pricing_category
0,27216,10.12.2021,SCAC81,Jan De Rijk,1708024,23020651,NL,2231,Rijnsburg,Alutech Rijnsburg,NL,8041 AL,Zwolle,Scania Production Zwolle,NL,2231,Rijnsburg,Alutech Rijnsburg,NL,8041 AL,Zwolle,Scania Production Zwolle,08.12.2021,08.12.2021,2560.0,78.720,13.600,FTL,SCAPAD,Only Inbound Invoicing Users - Price Adjustment,NaN,1206.85,not_lm_kg,0.0000,not_lm_kg
1,27034,03.12.2021,SCAC10-13,DSV ES,1695101,30239604,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,12540,VILLARREAL (CASTELLÓN),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,12540,VILLARREAL (CASTELLÓN),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",29.11.2021,24.11.2021,12845.0,69.750,11.800,LTL,SCAMAT,Maut,23226.8835 kg.,92.90,kg,23226.8835,23000_24499_kg
2,27034,03.12.2021,SCAC10-13,DSV ES,1695100,30239615,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,29.11.2021,24.11.2021,13812.0,72.853,13.262,LTL,SCAMAT,Maut,24499.0 kg.,92.90,kg,24499.0000,23000_24499_kg
3,27034,03.12.2021,SCAC10-13,DSV ES,1700008,30240209,ES,19004,GUADALAJARA,"NEOVIA LOGISTICS SERVICES SPAIN, S.A.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,19004,GUADALAJARA,"NEOVIA LOGISTICS SERVICES SPAIN, S.A.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,03.12.2021,30.11.2021,14772.0,71.332,13.600,LTL,SCAMAT,Maut,24499.0 kg.,92.90,kg,24499.0000,23000_24499_kg
4,27034,03.12.2021,SCAC10-13,DSV ES,1699921,30240157,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,03.12.2021,30.11.2021,8654.0,75.361,13.304,LTL,SCAMAT,Maut,24499.0 kg.,92.90,kg,24499.0000,23000_24499_kg


In [18]:
count_from = df.groupby(['Load no'],as_index=False).agg(
                    From_country=('Country TO consignor','nunique')
                    ,From_city=('City TO consignor','nunique'))


count_to = df.groupby(['Load no'],as_index=False).agg(
                    To_country=('Country TO consignee','nunique')
                    ,To_city=('City TO consignee','nunique'))

In [19]:
df_g = df.groupby(['Load no','Country TO consignor','City TO consignor','Cost desc'], 
           as_index=False).agg(
           To_country=('Country TO consignee',np.unique),
           To_city=('City TO consignee','unique'),
           
           leg_consignor_city=('City leg consignor',np.unique),
           leg_consignor_name=('Name leg consignor',np.unique),    
           leg_consignee_city=('City leg consignee',np.unique),
           leg_consignee_name=('Name leg consignee',np.unique),    
    
    
    
           Transpot_count=('TO no','nunique'),
           Bill_count=('Bill no','nunique'),
           transporter=('Bill sender id','max'),
           transporter_name=('Bill sender name','max'),
           pickup_date=('Pickup date','max'),
          weight_kg=('Weight TO [kg]','unique'),
          volumn_m3=('Volume TO [m3]','unique'),
          loadind_meter=('Loading meter leg','unique'),
           transport_mode=('Transport mode leg','max'),
           pricing_category=('pricing_category','max'),
           uom_type=('uom_type','max'),
           uom_value=('uom_value','unique'),
           amount=('Amount','sum'))

In [20]:
df_g[df_g['Load no']==1721035]

,Load no,Country TO consignor,City TO consignor,Cost desc,To_country,To_city,leg_consignor_city,leg_consignor_name,leg_consignee_city,leg_consignee_name,Transpot_count,Bill_count,transporter,transporter_name,pickup_date,weight_kg,volumn_m3,loadind_meter,transport_mode,pricing_category,uom_type,uom_value,amount
164805,1721035,ES,Burgos,Freight Charges,[SE],[OSKARSHAMN],[IRUN],[Schenker ES (ES) X-DOCK IRUN],[LIEUSAINT],[BREGER (FR) X-DOCK LIEUSAINT],1,1,SCAC26,BREGER,30.12.2021,[96.0],[0.143],[0.023],LTL,9_lm,lm,[8.6234],6.10
164806,1721035,ES,Burgos,Fuel Surcharges,[SE],[OSKARSHAMN],[IRUN],[Schenker ES (ES) X-DOCK IRUN],[LIEUSAINT],[BREGER (FR) X-DOCK LIEUSAINT],1,1,SCAC26,BREGER,30.12.2021,[96.0],[0.143],[0.023],LTL,not_lm_kg,not_lm_kg,[0.0],0.23
164807,1721035,ES,Burgos,Handling Cost (X-dock),[SE],[OSKARSHAMN],[IRUN],[Schenker ES (ES) X-DOCK IRUN],[LIEUSAINT],[BREGER (FR) X-DOCK LIEUSAINT],1,1,SCAC26-2,BREGER X-dock Luisant,30.12.2021,[96.0],[0.143],[0.023],LTL,not_lm_kg,not_lm_kg,[0.0],0.91
164808,1721035,ES,ERMUA,Freight Charges,"[BE, NL, SE]","[SODERTALJE, Hasselt, OUDSBERGEN , OSKARSHAMN]",[IRUN],[Schenker ES (ES) X-DOCK IRUN],[LIEUSAINT],[BREGER (FR) X-DOCK LIEUSAINT],10,1,SCAC26,BREGER,30.12.2021,"[689.0, 263.0, 1676.0, 2318.0, 380.0, 92.0, 49...","[1.71, 1.021, 5.837, 8.499, 0.48, 0.31, 1.4, 1...","[0.28, 0.171, 1.017, 1.457, 0.077, 0.05, 0.244...",LTL,9_lm,lm,[8.6234],563.18
164809,1721035,ES,ERMUA,Fuel Surcharges,"[BE, NL, SE]","[SODERTALJE, Hasselt, OUDSBERGEN , OSKARSHAMN]",[IRUN],[Schenker ES (ES) X-DOCK IRUN],[LIEUSAINT],[BREGER (FR) X-DOCK LIEUSAINT],10,1,SCAC26,BREGER,30.12.2021,"[689.0, 263.0, 1676.0, 2318.0, 380.0, 92.0, 49...","[1.71, 1.021, 5.837, 8.499, 0.48, 0.31, 1.4, 1...","[0.28, 0.171, 1.017, 1.457, 0.077, 0.05, 0.244...",LTL,not_lm_kg,not_lm_kg,[0.0],20.87
164810,1721035,ES,ERMUA,Handling Cost (X-dock),"[BE, NL, SE]","[SODERTALJE, Hasselt, OSKARSHAMN, OUDSBERGEN ]",[IRUN],[Schenker ES (ES) X-DOCK IRUN],[LIEUSAINT],[BREGER (FR) X-DOCK LIEUSAINT],10,1,SCAC26-2,BREGER X-dock Luisant,30.12.2021,"[689.0, 1676.0, 263.0, 261.0, 1445.0, 380.0, 4...","[1.71, 5.837, 1.021, 1.66, 4.448, 0.48, 0.139,...","[0.28, 1.017, 0.171, 0.305, 0.773, 0.077, 0.02...",LTL,not_lm_kg,not_lm_kg,[0.0],84.26
164811,1721035,ES,PAMPLONA,Freight Charges,[SE],[SODERTALJE],[IRUN],[Schenker ES (ES) X-DOCK IRUN],[LIEUSAINT],[BREGER (FR) X-DOCK LIEUSAINT],1,1,SCAC26,BREGER,30.12.2021,[544.0],[1.12],[0.212],LTL,9_lm,lm,[8.6234],34.59
164812,1721035,ES,PAMPLONA,Fuel Surcharges,[SE],[SODERTALJE],[IRUN],[Schenker ES (ES) X-DOCK IRUN],[LIEUSAINT],[BREGER (FR) X-DOCK LIEUSAINT],1,1,SCAC26,BREGER,30.12.2021,[544.0],[1.12],[0.212],LTL,not_lm_kg,not_lm_kg,[0.0],1.28
164813,1721035,ES,PAMPLONA,Handling Cost (X-dock),[SE],[SODERTALJE],[IRUN],[Schenker ES (ES) X-DOCK IRUN],[LIEUSAINT],[BREGER (FR) X-DOCK LIEUSAINT],1,1,SCAC26-2,BREGER X-dock Luisant,30.12.2021,[544.0],[1.12],[0.212],LTL,not_lm_kg,not_lm_kg,[0.0],5.17
164814,1721035,ES,"VILASSAR DE DALT, BARCELONA",Freight Charges,[SE],[SODERTALJE],[IRUN],[Schenker ES (ES) X-DOCK IRUN],[LIEUSAINT],[BREGER (FR) X-DOCK LIEUSAINT],1,1,SCAC26,BREGER,30.12.2021,[101.0],[1.402],[0.257],LTL,9_lm,lm,[8.6234],29.68


In [21]:
df_g[(df_g['Load no']==1706034) & (df_g['Cost desc']=='Freight Charges') ]

,Load no,Country TO consignor,City TO consignor,Cost desc,To_country,To_city,leg_consignor_city,leg_consignor_name,leg_consignee_city,leg_consignee_name,Transpot_count,Bill_count,transporter,transporter_name,pickup_date,weight_kg,volumn_m3,loadind_meter,transport_mode,pricing_category,uom_type,uom_value,amount
79986,1706034,TR,BURSA,Freight Charges,[PL],[SLUPSK],[ISTANBUL],[EKOL (TR) X-DOCK GEBZE],[HERNE],[DUVENBECK (DE) X-DOCK HERNE],1,2,SCAC31-1,EKOL,08.12.2021,[175.0],[0.576],[0.1],MR,150_199_kg,lm,"[191.808, 10.266]",24.07
79990,1706034,TR,BZE/KOCAELI,Freight Charges,[PL],[SLUPSK],[ISTANBUL],[EKOL X-DOCK GEBZE],[HERNE],[DUVENBECK (DE) X-DOCK HERNE],2,2,SCAC31-1,EKOL,08.12.2021,"[2385.0, 1418.0]","[18.06, 12.03]","[5.219, 3.513]",MR,9000_9999_kg,lm,"[9655.15, 6499.05, 10.266]",1976.34
79994,1706034,TR,CAYIROVA-GEBZE,Freight Charges,[NL],[Hasselt],[ISTANBUL],[EKOL X-DOCK GEBZE],[Hasselt],[Scania CV AB],1,1,SCAC31-1,EKOL,08.12.2021,[142.0],[1.921],[0.272],MR,11_lm,lm,[10.266],74.85
79998,1706034,TR,GEBZE - KOCAELI,Freight Charges,[NL],[Hasselt],[ISTANBUL],[EKOL X-DOCK GEBZE],[Hasselt],[Scania CV AB],1,1,SCAC31-1,EKOL,08.12.2021,[1356.0],[3.236],[0.487],MR,11_lm,lm,[10.266],158.66
80002,1706034,TR,KOCAELI,Freight Charges,[NL],[Hasselt],[ISTANBUL],[EKOL (TR) X-DOCK GEBZE],[Hasselt],[Scania CV AB],1,1,SCAC31-1,EKOL,08.12.2021,[1782.0],[1.848],[0.275],MR,11_lm,lm,[10.266],208.50
80006,1706034,TR,RSA,Freight Charges,[PL],[SLUPSK],[ISTANBUL],[EKOL X-DOCK GEBZE],[HERNE],[DUVENBECK (DE) X-DOCK HERNE],1,2,SCAC31-1,EKOL,08.12.2021,[150.0],[1.44],[0.4],MR,700_799_kg,lm,"[740.0, 10.266]",90.53


In [24]:
col_to_keep = ['Load no', 'Country TO consignor', 'City TO consignor', 'Cost desc', 'To_country', 'To_city', 
               'leg_consignor_city','leg_consignor_name','leg_consignee_city','leg_consignee_name',
               'Transpot_count','Bill_count','pricing_category','amount']

df_g_filtered = df_g[col_to_keep]


df_g_filtered.head(2)

,Load no,Country TO consignor,City TO consignor,Cost desc,To_country,To_city,leg_consignor_city,leg_consignor_name,leg_consignee_city,leg_consignee_name,Transpot_count,Bill_count,pricing_category,amount
0,1428428,SE,Järna,Freight Charges,[SE],[KALIX],[Järna],[VIDA PACKAGING LOGISTICS AB],[KALIX],[ANVA COMPONENTS AB],1,1,16000_16999_kg,1355.56
1,1428428,SE,Järna,Fuel Surcharges,[SE],[KALIX],[Järna],[VIDA PACKAGING LOGISTICS AB],[KALIX],[ANVA COMPONENTS AB],1,1,not_lm_kg,6.96


In [28]:
df_split = pd.concat([df_g_filtered,pd.DataFrame(df_g_filtered['To_country'].to_list())], axis=1)



df_split.columns= ['Load no', 'Country TO consignor', 'City TO consignor', 'Cost desc', 'To_country', 'To_city', 
               'leg_consignor_city','leg_consignor_name','leg_consignee_city','leg_consignee_name',
               'Transpot_count','Bill_count','pricing_category','amount', 'dist_country_1', 'dist_country_2','dist_country_3', 
                    'dist_country_4', 'dist_country_5']


df_final = pd.concat([df_split,pd.DataFrame(df_split['To_city'].to_list())], axis=1)

df_final.columns=['Load no', 'Country TO consignor', 'City TO consignor',
       'Cost desc', 'To_country', 'To_city', 'leg_consignor_city',
       'leg_consignor_name', 'leg_consignee_city', 'leg_consignee_name',
       'Transpot_count', 'Bill_count', 'pricing_category', 'amount',
       'dist_country_1', 'dist_country_2', 'dist_country_3','dist_country_4', 'dist_country_5',
        'dist_country_1', 'dist_country_2','dist_country_3','dist_country_4', 'dist_country_5', 
        'dist_city_6', 'dist_city_7', 'dist_city_8', 'dist_city_9', 'dist_city_10', 'dist_city_11']

df_final.head(2)


,Load no,Country TO consignor,City TO consignor,Cost desc,To_country,To_city,leg_consignor_city,leg_consignor_name,leg_consignee_city,leg_consignee_name,Transpot_count,Bill_count,pricing_category,amount,dist_country_1,dist_country_2,dist_country_3,dist_country_4,dist_country_5,dist_country_1,dist_country_2,dist_country_3,dist_country_4,dist_country_5,dist_city_6,dist_city_7,dist_city_8,dist_city_9,dist_city_10,dist_city_11
0,1428428,SE,Järna,Freight Charges,[SE],[KALIX],[Järna],[VIDA PACKAGING LOGISTICS AB],[KALIX],[ANVA COMPONENTS AB],1,1,16000_16999_kg,1355.56,SE,None,None,None,None,KALIX,None,None,None,None,None,None,None,None,None,None
1,1428428,SE,Järna,Fuel Surcharges,[SE],[KALIX],[Järna],[VIDA PACKAGING LOGISTICS AB],[KALIX],[ANVA COMPONENTS AB],1,1,not_lm_kg,6.96,SE,None,None,None,None,KALIX,None,None,None,None,None,None,None,None,None,None


In [29]:
df_final = df_final.drop(columns=['To_country','To_city'])

In [30]:
df_final = df_final.fillna(value=0)

In [31]:
df_final.head(2)

,Load no,Country TO consignor,City TO consignor,Cost desc,leg_consignor_city,leg_consignor_name,leg_consignee_city,leg_consignee_name,Transpot_count,Bill_count,pricing_category,amount,dist_country_1,dist_country_2,dist_country_3,dist_country_4,dist_country_5,dist_country_1,dist_country_2,dist_country_3,dist_country_4,dist_country_5,dist_city_6,dist_city_7,dist_city_8,dist_city_9,dist_city_10,dist_city_11
0,1428428,SE,Järna,Freight Charges,[Järna],[VIDA PACKAGING LOGISTICS AB],[KALIX],[ANVA COMPONENTS AB],1,1,16000_16999_kg,1355.56,SE,0,0,0,0,KALIX,0,0,0,0,0,0,0,0,0,0
1,1428428,SE,Järna,Fuel Surcharges,[Järna],[VIDA PACKAGING LOGISTICS AB],[KALIX],[ANVA COMPONENTS AB],1,1,not_lm_kg,6.96,SE,0,0,0,0,KALIX,0,0,0,0,0,0,0,0,0,0


In [34]:
df_split2 = pd.concat([df_final,pd.DataFrame(df_final['leg_consignor_city'].to_list())], axis=1)

df_split2.columns= ['Load no', 'Country TO consignor', 'City TO consignor',
                   'Cost desc', 'leg_consignor_city', 'leg_consignor_name',
                   'leg_consignee_city', 'leg_consignee_name', 'Transpot_count',
                   'Bill_count', 'pricing_category', 'amount', 'dist_country_1',
                   'dist_country_2', 'dist_country_3', 'dist_country_4',
                   'dist_country_5', 'dist_country_1', 'dist_country_2',
                   'dist_country_3', 'dist_country_4', 'dist_country_5',
                   'dist_city_6', 'dist_city_7', 'dist_city_8', 'dist_city_9',
                   'dist_city_10', 'dist_city_11', 'leg_consignor_city']

In [38]:
df_split3 = pd.concat([df_split2,pd.DataFrame(df_split2['leg_consignor_name'].to_list())], axis=1)

df_split3.columns= ['Load no', 'Country TO consignor', 'City TO consignor',
                   'Cost desc', 'leg_consignor_city', 'leg_consignor_name',
                   'leg_consignee_city', 'leg_consignee_name', 'Transpot_count',
                   'Bill_count', 'pricing_category', 'amount', 'dist_country_1',
                   'dist_country_2', 'dist_country_3', 'dist_country_4',
                   'dist_country_5', 'dist_country_1', 'dist_country_2',
                   'dist_country_3', 'dist_country_4', 'dist_country_5',
                   'dist_city_6', 'dist_city_7', 'dist_city_8', 'dist_city_9',
                   'dist_city_10', 'dist_city_11', 'leg_consignor_city', 'leg_consignor_name_1', 'leg_consignor_name_2', 
                    'leg_consignor_name_3', 'leg_consignor_name_4']

In [43]:
df_split4 = pd.concat([df_split3,pd.DataFrame(df_split3['leg_consignee_city'].to_list())], axis=1)

df_split4.columns= ['Load no', 'Country TO consignor', 'City TO consignor',
                   'Cost desc', 'leg_consignor_city', 'leg_consignor_name',
                   'leg_consignee_city', 'leg_consignee_name', 'Transpot_count',
                   'Bill_count', 'pricing_category', 'amount', 'dist_country_1',
                   'dist_country_2', 'dist_country_3', 'dist_country_4',
                   'dist_country_5', 'dist_country_1', 'dist_country_2',
                   'dist_country_3', 'dist_country_4', 'dist_country_5',
                   'dist_city_6', 'dist_city_7', 'dist_city_8', 'dist_city_9',
                   'dist_city_10', 'dist_city_11', 'leg_consignor_city',
                   'leg_consignor_name_1', 'leg_consignor_name_2',
                   'leg_consignor_name_3', 'leg_consignor_name_4', 'leg_consignee_city_1', 
                    'leg_consignee_city_2', 'leg_consignee_city_3', 'leg_consignee_city_4', 'leg_consignee_city_5']

In [48]:
df_final4 = pd.concat([df_split4,pd.DataFrame(df_split4['leg_consignee_name'].to_list())], axis=1)

df_final4.columns= ['Load no', 'Country TO consignor', 'City TO consignor',
                   'Cost desc', 'leg_consignor_city', 'leg_consignor_name',
                   'leg_consignee_city', 'leg_consignee_name', 'Transpot_count',
                   'Bill_count', 'pricing_category', 'amount', 'dist_country_1',
                   'dist_country_2', 'dist_country_3', 'dist_country_4',
                   'dist_country_5', 'dist_country_1', 'dist_country_2',
                   'dist_country_3', 'dist_country_4', 'dist_country_5',
                   'dist_city_6', 'dist_city_7', 'dist_city_8', 'dist_city_9',
                   'dist_city_10', 'dist_city_11', 'leg_consignor_city',
                   'leg_consignor_name_1', 'leg_consignor_name_2',
                   'leg_consignor_name_3', 'leg_consignor_name_4',
                   'leg_consignee_city_1', 'leg_consignee_city_2',
                   'leg_consignee_city_3', 'leg_consignee_city_4',
                   'leg_consignee_city_5', 'leg_consignee_name_1', 'leg_consignee_name_2', 
                    'leg_consignee_name_3', 'leg_consignee_name_4', 'leg_consignee_name_5']

In [50]:
df_transposed= df_final4.drop(columns=['leg_consignor_city','leg_consignor_name','leg_consignee_city','leg_consignee_name'])

df_transposed.fillna(0,inplace=True)

In [51]:
df_transposed.head()

,Load no,Country TO consignor,City TO consignor,Cost desc,Transpot_count,Bill_count,pricing_category,amount,dist_country_1,dist_country_2,dist_country_3,dist_country_4,dist_country_5,dist_country_1,dist_country_2,dist_country_3,dist_country_4,dist_country_5,dist_city_6,dist_city_7,dist_city_8,dist_city_9,dist_city_10,dist_city_11,leg_consignor_name_1,leg_consignor_name_2,leg_consignor_name_3,leg_consignor_name_4,leg_consignee_city_1,leg_consignee_city_2,leg_consignee_city_3,leg_consignee_city_4,leg_consignee_city_5,leg_consignee_name_1,leg_consignee_name_2,leg_consignee_name_3,leg_consignee_name_4,leg_consignee_name_5
0,1428428,SE,Järna,Freight Charges,1,1,16000_16999_kg,1355.56,SE,0,0,0,0,KALIX,0,0,0,0,0,0,0,0,0,0,VIDA PACKAGING LOGISTICS AB,0,0,0,KALIX,0,0,0,0,ANVA COMPONENTS AB,0,0,0,0
1,1428428,SE,Järna,Fuel Surcharges,1,1,not_lm_kg,6.96,SE,0,0,0,0,KALIX,0,0,0,0,0,0,0,0,0,0,VIDA PACKAGING LOGISTICS AB,0,0,0,KALIX,0,0,0,0,ANVA COMPONENTS AB,0,0,0,0
2,1535615,DE,BAD KISSINGEN,Waiting time (consignor),3,1,not_lm_kg,96.00,NL,0,0,0,0,Zwolle,Hasselt,0,0,0,0,0,0,0,0,0,JOYSON SAFETY SYSTEMS PLASTEC GMBH,0,0,0,Hasselt,Zwolle,0,0,0,Scania CV AB,Scania Production Zwolle,0,0,0
3,1535615,DE,BAD KISSINGEN,Waiting time Supplier,3,1,not_lm_kg,203.50,NL,0,0,0,0,Hasselt,Zwolle,0,0,0,0,0,0,0,0,0,JOYSON SAFETY SYSTEMS PLASTEC GMBH,0,0,0,Hasselt,Zwolle,0,0,0,Scania CV AB,Scania Production Zwolle,0,0,0
4,1537055,DE,BAD KISSINGEN,Waiting time (consignor),2,1,not_lm_kg,162.50,NL,0,0,0,0,Meppel,Zwolle,0,0,0,0,0,0,0,0,0,JOYSON SAFETY SYSTEMS PLASTEC GMBH,0,0,0,Meppel,Zwolle,0,0,0,Scania Production Meppel,Scania Production Zwolle,0,0,0


In [52]:
df_transposed.to_excel('dec_grouped1.xlsx',index=False)